In [185]:
glsl=[]
with open("D:/GitHub/Bilibili_Anime4K/Anime4K_Upscale_CNN_M_x2_Deblur.glsl", "r") as f:
    glsl=f.readlines()

In [191]:
frags=[]
buffer=[]

for idx, line in enumerate(glsl):
    if "//!DESC" in line or idx == len(glsl) - 1:
        if "//!DESC" not in line:
            buffer.append(line)
        if "//!DESC" in buffer[0]:
            frags.append(buffer)
        buffer=[]
    buffer.append(line)

In [192]:
def tex(name):
    return "vec4 " + name + "_tex(vec2 pos) {\n"+\
        "    return texture2D(" + name + ", pos);\n"+\
        "}\n"

for i in range(len(frags)):
    defines={}
    hook_name=""
    is_in_body=False
    
    global_variables=[]
    frag_names=[]
    frag_signatures=[]

    for j in range(len(frags[i])):
        if "//!" in frags[i][j]:
            if "DESC" in frags[i][j]:
                hook_name=frags[i][j].split()[-1]
                frag_names.append(hook_name)
            if "HOOK" in frags[i][j] or "BIND" in frags[i][j]:
                global_variables.append(frags[i][j].split()[-1])
            continue
            
        if "#define" in frags[i][j]:
            defines[frags[i][j].split()[1]] = frags[i][j].split()[2]
            frags[i][j] = "\n"
            continue
        
        for origin, replacement in defines.items():
            frags[i][j] = frags[i][j].replace(origin, replacement)
        
        if "vec4 hook()" in frags[i][j]:
            is_in_body = True
            type_name = "void"

            frags[i][j] = frags[i][j].replace("vec4 hook()", type_name + " main()")
            frag_signatures.append(type_name + " main()")
        
        if is_in_body:
            frags[i][j] = frags[i][j].replace("return", "gl_FragColor =")

    global_variables=sorted(set(global_variables))
    print(global_variables)

    generated_global_declarations=[]
    generated_global_helper=[]

    for j in global_variables:
        generated_global_declarations.append("uniform sampler2D " + j + ";\n")
        generated_global_helper.append(tex(j))

    generated_global_declarations.append("uniform vec2 u_size;\nvarying vec2 v_tex_pos;\n")

    generated_shader = [] + ["precision mediump float;\n"] + generated_global_declarations + ["\n"] + generated_global_helper + ["\n"] + frags[i]

    with open("D:/GitHub/Bilibili_Anime4K/Frags/" + str(i) + ".glsl", "w") as f:
        f.writelines(generated_shader)

['HOOKED', 'NATIVE']
['HOOKED', 'LUMAN0', 'NATIVE']
['HOOKED', 'LUMAN1', 'NATIVE']
['HOOKED', 'LUMAN2', 'NATIVE']
['HOOKED', 'LUMAN3', 'NATIVE']
['HOOKED', 'LUMAN4', 'NATIVE']
['HOOKED', 'LUMAN1', 'LUMAN2', 'LUMAN3', 'LUMAN4', 'LUMAN5', 'NATIVE']
['HOOKED', 'NATIVE']
['HOOKED', 'MMKERNEL', 'NATIVE']
['HOOKED', 'LUMAN0', 'MMKERNEL', 'NATIVE']
